<a href="https://colab.research.google.com/github/danilinekamgue/python/blob/master/Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning in Python 

If you don't have a proper hardware / software configuration you can use Colab: https://colab.research.google.com/notebooks/intro.ipynb?hl=nb

Many libraries are already installed (PyTorch, Tensorflow...)

Libraries we will use throughout these tutorials:
* numpy
* pandas
* matplotlib + seaborn
* scikit-learn
* pytorch + torchvision + tensorboard

## Split dataset

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Create fake dataset with numbers from 1 to 200
x_tosplit = np.arange(1,201,1).reshape(100,2)
# Create random targets (0 or 1) for classification problem
y_tosplit = np.random.randint(0,2, (x_tosplit.shape[0]))
print(x_tosplit.shape)
print(y_tosplit.shape)

In [ ]:
# Split dataset with 70% training, 30% test, balancing classes
x_train, x_test, y_train, y_test = train_test_split(
    x_tosplit, y_tosplit, train_size=0.7, shuffle=True, stratify=y_tosplit)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_test)

## Perceptron

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import pandas as pd

In [ ]:
# Read data from file
data = pd.read_csv('perceptron-data.csv')
labels = torch.tensor(data['target'].values, dtype=torch.float32)
data = torch.tensor(data[['x', 'y']].values, dtype=torch.float32)
plt.scatter(data[:, 0], data[:, 1], c=labels)

In [ ]:
accuracies = []
epochs = 100 # how many times we want to see all dataset
eta = 0.1 # learning rate

numpt = data.size(0) # number of patterns
inputDim = data.size(1)

# perceptron is sign(wx+b)
weights = torch.randn(inputDim, dtype=torch.float32) # w
bias = torch.zeros(1) # b


for epoch in range(epochs): # for each epoch
    total_accuracy = 0

    for idx in range(numpt): # for each pattern in the dataset
        X = data[idx,:]
        y = labels[idx]

        # wx+b
        out = torch.add(torch.dot(weights, X), bias).item()
        # sign(out)
        out = 1 if out > 0 else 0
        
        # if classification is correct increase accuracy
        if out == y:
            total_accuracy += 1
        
        # update weights and bias with perceptron learning rule
        weights += eta * (y - out) * X
        bias += eta * (y - out)        

    accuracies.append(total_accuracy / float(numpt))

    
# visualize result

print(f'Last accuracy: {accuracies[-1]*100}%')
# plot points, hyperplane and learning curve
plt.figure()
plt.scatter(data[:,0].numpy(), data[:,1].numpy(), c=labels.numpy())
xr = np.linspace(0, 20, 10)
yr = (-1 / weights[1].item()) * (weights[0].item() * xr  + bias.item())
plt.plot(xr, yr,'-')
plt.xlim(-1, 21)
plt.ylim(-3, 22)
plt.show()

plt.figure()
plt.plot(accuracies, '-')
plt.show()

## Multi Layer Perceptron (a.k.a. feedforward networks)

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
class MLP(nn.Module):
    def __init__(self, layers):
        """
        :param layers: a list with as many elements as the number of hidden layers.
            Each element specifies the number of units in that hidden layer.
            e.g. [input_size, 64, 128, output_size]
        """

        super(MLP, self).__init__()

        assert len(layers) >= 2, "Layers must specify at least input and output size."
        
        # create neural network layers
        mlp_layers = []
        for i in range(1, len(layers)):
            # add a Linear layer 
            mlp_layers.append(nn.Linear(layers[i-1], layers[i], bias=True))
            # add ReLU activation function
            mlp_layers.append(nn.ReLU())
        
        # put all layers together
        self.model = nn.Sequential(*mlp_layers)

        
    def forward(self, x):
        """
        This method is called each time you invoke your instance of this class.
        e.g. mlp = MLP([10,64,3])
            mlp(x) -> will call this method
        """
        
        out = self.model(x) # compute output of neural network
        return out

In [ ]:
# Read data from file
data = pd.read_csv('perceptron-data_notsep.csv')
# put data in pytorch tensors
labels = torch.tensor(data['target'].values).long() # transform labels from {-1, 1} to {0, 1}
data = torch.tensor(data[['x', 'y']].values, dtype=torch.float32)
# create a dataloader to automatically shuffle and get batches of data
dataset = torch.utils.data.TensorDataset(data, labels)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True, drop_last=True)
numpt = data.size(0)
inputDim = data.size(1)


In [ ]:
mlp = MLP([inputDim, 64, 2]) # create neural network

# the optimizer is responsible for the updating of network's parameters
optimizer = optim.Adam(mlp.parameters(), lr=1e-3, weight_decay=1e-4)
# the error / cost function
criterion = nn.CrossEntropyLoss()
epochs = 200 # how many times to loop over the dataset

losses = []
accuracies = []
for epoch in range(epochs):
    total_loss, total_accuracy = 0, 0
    for x,y in dataloader: # for each batch of data        
        optimizer.zero_grad() # reset gradients, to avoid using previous ones
        out = mlp(x) # compute output
        loss = criterion(out, y) # compute error
        loss.backward() # compute gradients
        optimizer.step() # update parameters
        
        # compute accuracy
        with torch.no_grad():
            accuracy = (out.argmax(dim=1) == y).sum() / float(y.numel())
        total_loss += loss.item()
        total_accuracy += accuracy.item()
        
    losses.append(total_loss / float(len(dataloader)))
    accuracies.append(total_accuracy / float(len(dataloader)))


print(f"Last accuracy: {accuracies[-1]*100}%")
plt.figure()
plt.plot(losses, '-')
plt.show()
plt.figure()
plt.plot(accuracies, '-')
plt.show()

## Some popular metrics (and combinations of them)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, \
    precision_recall_curve, plot_precision_recall_curve
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

In [ ]:
X, y = make_classification(n_classes=2, n_informative=6) # build fake datasset
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True) # split dataset
model = LogisticRegression().fit(x_train, y_train) # create a model
predictions = model.predict(x_test) # predict on test set

In [ ]:
# F1
f1_score(y_test, predictions, average='macro')

In [ ]:
# Confusion matrix
confusion_matrix(y_test, predictions, normalize='all')

In [ ]:
# P-R curve
probs = model.decision_function(x_test)
p, r, t = precision_recall_curve(y_test, probs)
print(p)
print(r)
print(t)

In [ ]:
plt.plot(r,p)
plt.show()

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

In [ ]:
# what about ROC curve? Try it out!

## What is tensorboard?

Why tensorboard? Well... when you cannot debug normally...

but also for visualization :)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

In [ ]:
!ls runs

In [ ]:
for i in range(20):
    writer.add_scalar('loss', i,i)

In [ ]:
for i in range(20):
    writer.add_scalar('loss', i*i,i)

set x axis to step, otherwise you align results according to execution time

In [ ]:
for i in range(100):
    writer.add_scalars('functions', {'xsinx':i,
                                    'xcosx':i**2,
                                    'tanx': np.log(i+1)
    }, i)